##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TPU embedding_columns에서 TPUEmbedding 레이어로 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/tpu_embedding"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/tpu_embedding.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/tpu_embedding.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/tpu_embedding.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

이 가이드는 `TPUEstimator`를 사용하는 TensorFlow 1의 `embedding_column` API에서 `TPUStrategy`를 사용하는 TensorFlow 2의 `TPUEmbedding` 레이어 API로 [TPU](../../guide/tpu.ipynb)에 대한 임베딩 훈련을 마이그레이션하는 방법을 설명합니다.

임베딩은 (대형) 행렬입니다. 임베딩은 희소 특성 공간에서 밀집 벡터로 매핑하는 조회 테이블입니다. 임베딩은 특성 사이의 복잡한 유사성과 관계를 캡처하여 효율적이고 밀도 높은 표현을 제공합니다.

TensorFlow에는 TPU의 훈련 임베딩용 전문 지원이 포함되어 있습니다. 이 TPU 전용 임베딩 지원을 통해 단일 TPU 기기의 메모리보다 큰 임베딩을 훈련하고 TPU에서 희소한 비정형 입력을 사용할 수 있습니다.

- TensorFlow 1에서 `tf.compat.v1.estimator.tpu.TPUEstimator`는 TPU로 서비스를 제공하기 위해 훈련, 평가, 예측 및 내보내기를 캡슐화하는 고수준의 API입니다. `tf.compat.v1.tpu.experimental.embedding_column`을 특별히 지원합니다.
- TensorFlow 2에서 이를 구현하려면 TensorFlow Recommenders의 `tfrs.layers.embedding.TPUEmbedding` 레이어를 사용하면 됩니다. 훈련하고 평가하려면 TPU 배포 전략인 `tf.distribute.TPUStrategy`을 사용합니다. 이 전략은 모델 구축(`tf.keras.Model`), 옵티마이저(`tf.keras.optimizers.Optimizer`), `Model.fit`를 사용하는 훈련, `tf.function`와 `tf.GradientTape`를 사용하는 사용자 정의 훈련 루프와 같은 Keras API와도 호환됩니다.

추가 정보는 `tfrs.layers.embedding.TPUEmbedding` 레이어의 API 문서와 `tf.tpu.experimental.embedding.TableConfig` 및 `tf.tpu.experimental.embedding.FeatureConfig` 문서를 참조하세요. `tf.distribute.TPUStrategy`의 개요는 [분산 훈련](../../guide/distributed_training.ipynb) 가이드 및 [TPU 사용](../../guide/tpu.ipynb) 가이드를 확인하세요. `TPUEstimator`에서 `TPUStrategy`로 마이그레이션하는 경우 [TPU 마이그레이 가이드](tpu_estimator.ipynb)를 확인하세요.

## 설치하기

먼저 [TensorFlow Recommenders](https://www.tensorflow.org/recommenders)를 설치하고 몇 가지 필수 패키지를 가져옵니다.

In [ ]:
!pip install tensorflow-recommenders

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

# TPUEmbedding layer is not part of TensorFlow.
import tensorflow_recommenders as tfrs

데모를 위해 간단한 데이터세트를 준비합니다.

In [ ]:
features = [[1., 1.5]]
embedding_features_indices = [[0, 0], [0, 1]]
embedding_features_values = [0, 5]
labels = [[0.3]]
eval_features = [[4., 4.5]]
eval_embedding_features_indices = [[0, 0], [0, 1]]
eval_embedding_features_values = [4, 3]
eval_labels = [[0.8]]

## TensorFlow 1: TPUEstimator를 사용하여 TPU에서 임베딩 훈련하기

TensorFlow 1에서는 `tf.compat.v1.tpu.experimental.embedding_column` API를 사용하여 TPU 임베딩을 설정하고 `tf.compat.v1.estimator.tpu.TPUEstimator`를 사용하여 TPU의 모델을 훈련/평가합니다.

입력은 0부터 시작되는 TPU 임베딩 테이블의 어휘 크기 범위를 나타내는 정수입니다. `tf.feature_column.categorical_column_with_identity`를 사용하여 입력을 범주형 ID로 인코딩하는 것으로 시작합니다. `num_buckets`은 임베딩 테이블(`10`)의 어휘 크기인 반면 입력 특성은 정수 값이므로 `key` 매개변수에 `"sparse_feature"`을 사용합니다.

In [ ]:
embedding_id_column = (
      tf1.feature_column.categorical_column_with_identity(
          key="sparse_feature", num_buckets=10))

다음으로 `tpu.experimental.embedding_column`을 사용하여 희소 범주형 입력을 밀집 표현으로 변환합니다. 여기서 `dimension`은 임베딩 테이블의 너비입니다. 이는 각 `num_buckets`에 대한 임베딩 벡터를 저장합니다.

In [ ]:
embedding_column = tf1.tpu.experimental.embedding_column(
    embedding_id_column, dimension=5)

이제 `tf.estimator.tpu.experimental.EmbeddingConfigSpec`을 통해 TPU 전용 임베딩 구성을 정의합니다. 나중에 이 구성을 `embedding_config_spec` 매개변수로 `tf.estimator.tpu.TPUEstimator`에 전달합니다.

In [ ]:
embedding_config_spec = tf1.estimator.tpu.experimental.EmbeddingConfigSpec(
    feature_columns=(embedding_column,),
    optimization_parameters=(
        tf1.tpu.experimental.AdagradParameters(0.05)))

다음으로 `TPUEstimator`를 사용하기 위해 다음을 정의합니다.

- 훈련 데이터용 입력 함수
- 평가 데이터용 평가 입력 함수
- `TPUEstimator`에 특성 및 라벨을 사용하여 훈련 연산을 정의하는 방법을 지시하는 모델 함수

In [ ]:
def _input_fn(params):
  dataset = tf1.data.Dataset.from_tensor_slices((
      {"dense_feature": features,
       "sparse_feature": tf1.SparseTensor(
           embedding_features_indices,
           embedding_features_values, [1, 2])},
           labels))
  dataset = dataset.repeat()
  return dataset.batch(params['batch_size'], drop_remainder=True)

def _eval_input_fn(params):
  dataset = tf1.data.Dataset.from_tensor_slices((
      {"dense_feature": eval_features,
       "sparse_feature": tf1.SparseTensor(
           eval_embedding_features_indices,
           eval_embedding_features_values, [1, 2])},
           eval_labels))
  dataset = dataset.repeat()
  return dataset.batch(params['batch_size'], drop_remainder=True)

def _model_fn(features, labels, mode, params):
  embedding_features = tf1.keras.layers.DenseFeatures(embedding_column)(features)
  concatenated_features = tf1.keras.layers.Concatenate(axis=1)(
      [embedding_features, features["dense_feature"]])
  logits = tf1.layers.Dense(1)(concatenated_features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  optimizer = tf1.tpu.CrossShardOptimizer(optimizer)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)

이러한 함수가 정의되면 클러스터 정보를 제공하는 `tf.distribute.cluster_resolver.TPUClusterResolver`와 `tf.compat.v1.estimator.tpu.RunConfig` 객체를 생성합니다.

정의한 모델 함수와 함께 이제 `TPUEstimator`를 생성할 수 있습니다. 이는 체크포인트 저장을 건너뛰기에 흐름이 단순합니다. 그런 다음 `TPUEstimator`의 훈련 및 평가 모두에 사용하는 배치 크기를 지정합니다.

In [ ]:
cluster_resolver = tf1.distribute.cluster_resolver.TPUClusterResolver(tpu='')
print("All devices: ", tf1.config.list_logical_devices('TPU'))

In [ ]:
tpu_config = tf1.estimator.tpu.TPUConfig(
    iterations_per_loop=10,
    per_host_input_for_training=tf1.estimator.tpu.InputPipelineConfig
          .PER_HOST_V2)
config = tf1.estimator.tpu.RunConfig(
    cluster=cluster_resolver,
    save_checkpoints_steps=None,
    tpu_config=tpu_config)
estimator = tf1.estimator.tpu.TPUEstimator(
    model_fn=_model_fn, config=config, train_batch_size=8, eval_batch_size=8,
    embedding_config_spec=embedding_config_spec)

모델 훈련을 시작하려면 `TPUEstimator.train`을 호출합니다.

In [ ]:
estimator.train(_input_fn, steps=1)

그런 다음 `TPUEstimator.evaluate`를 호출하여 평가 데이터를 사용하는 모델을 평가합니다.

In [ ]:
estimator.evaluate(_eval_input_fn, steps=1)

## TensorFlow 2: TPUStrategy를 사용하여 TPU에서 임베딩 훈련하기

TensorFlow 2에서 TPU 작업자를 훈련하려면 모델 정의 및 훈련/평가를 위해 Keras API와 `tf.distribute.TPUStrategy`를 사용해야 합니다(Keras Model.fit 및 사용자 정의 훈련 루프를 사용하는 더 많은 훈련 예제(`tf.function` 및 `tf.GradientTape` 포함)는 [TPU 사용](https://render.githubusercontent.com/guide/tpu.ipynb) 가이드 참조).

원격 클러스터에 연결하고 TPU 작업자를 초기화하려면 몇 가지 초기화 작업을 수행해야 하므로 먼저 `TPUClusterResolver`를 생성하여 클러스터 정보를 제공하고 클러스터에 연결합니다(자세한 내용은 [TPU 사용](../../guide/tpu.ipynb) 가이드의 *TPU 초기화* 섹션 참조).

In [ ]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

다음으로 데이터를 준비합니다. 이는 TensorFlow 1 예제에서 데이터세트를 생성한 방법과 유사하지만 데이터세트 함수가 이제 `params` 사전이 아닌 `tf.distribute.InputContext` 객체로 전달된다는 점이 다릅니다. 이 개체를 사용하여 로컬 배치 크기(및 데이터를 적절하게 분할할 수 있도록 이 파이프라인의 대상 호스트)를 결정할 수 있습니다.

- `tfrs.layers.embedding.TPUEmbedding` API를 사용하는 경우 `TPUEmbedding`에는 고정 배치 크기가 필요하기 때문에 `Dataset.batch`로 데이터세트를 배치 처리할 때에는 `drop_remainder=True` 옵션을 포함하는 것이 중요합니다.
- 또한, 동일한 기기 세트에서 작업을 수행하는 경우 평가 및 훈련에 동일한 배치 크기를 사용해야 합니다.
- 마지막으로 `tf.distribute.InputOptions`(전략별 구성 포함)에서 특수 입력 옵션인 `experimental_fetch_to_device=False`와 함께 `tf.keras.utils.experimental.DatasetCreator`를 사용해야 합니다. 이것은 아래에 설명되어 있습니다.

In [ ]:
global_batch_size = 8

def _input_dataset(context: tf.distribute.InputContext):
  dataset = tf.data.Dataset.from_tensor_slices((
      {"dense_feature": features,
       "sparse_feature": tf.SparseTensor(
           embedding_features_indices,
           embedding_features_values, [1, 2])},
           labels))
  dataset = dataset.shuffle(10).repeat()
  dataset = dataset.batch(
      context.get_per_replica_batch_size(global_batch_size),
      drop_remainder=True)
  return dataset.prefetch(2)

def _eval_dataset(context: tf.distribute.InputContext):
  dataset = tf.data.Dataset.from_tensor_slices((
      {"dense_feature": eval_features,
       "sparse_feature": tf.SparseTensor(
           eval_embedding_features_indices,
           eval_embedding_features_values, [1, 2])},
           eval_labels))
  dataset = dataset.repeat()
  dataset = dataset.batch(
      context.get_per_replica_batch_size(global_batch_size),
      drop_remainder=True)
  return dataset.prefetch(2)

input_options = tf.distribute.InputOptions(
    experimental_fetch_to_device=False)

input_dataset = tf.keras.utils.experimental.DatasetCreator(
    _input_dataset, input_options=input_options)

eval_dataset = tf.keras.utils.experimental.DatasetCreator(
    _eval_dataset, input_options=input_options)

다음으로, 데이터가 준비되면 `TPUStrategy`를 생성하고 이 전략(`Strategy.scope`)의 범위에서 모델, 메트릭, 옵티마이저를 정의합니다.

각 `tf.function` 호출 중에 실행할 배치의 수를 지정해야 하며 이는 성능에 중요하므로 `Model.compile`에서 `steps_per_execution`에 대한 숫자를 선택해야 합니다. 이 인수는 `TPUEstimator`에서 사용되는 `iterations_per_loop`과 유사합니다.

TensorFlow 1에서 `tf.tpu.experimental.embedding_column`(및 `tf.tpu.experimental.shared_embedding_column`)을 통해 지정한 특성 및 테이블 구성은 TensorFlow 2에서 구성 객체 쌍을 통해 직접 지정할 수 있습니다.

- `tf.tpu.experimental.embedding.FeatureConfig`
- `tf.tpu.experimental.embedding.TableConfig`

(자세한 내용은 관련 API 문서를 참조하세요.)

In [ ]:
strategy = tf.distribute.TPUStrategy(cluster_resolver)
with strategy.scope():
  if hasattr(tf.keras.optimizers, "legacy"):
    optimizer = tf.keras.optimizers.legacy.Adagrad(learning_rate=0.05)
  else:
    optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
  dense_input = tf.keras.Input(shape=(2,), dtype=tf.float32, batch_size=global_batch_size)
  sparse_input = tf.keras.Input(shape=(), dtype=tf.int32, batch_size=global_batch_size)
  embedded_input = tfrs.layers.embedding.TPUEmbedding(
      feature_config=tf.tpu.experimental.embedding.FeatureConfig(
          table=tf.tpu.experimental.embedding.TableConfig(
              vocabulary_size=10,
              dim=5,
              initializer=tf.initializers.TruncatedNormal(mean=0.0, stddev=1)),
          name="sparse_input"),
      optimizer=optimizer)(sparse_input)
  input = tf.keras.layers.Concatenate(axis=1)([dense_input, embedded_input])
  result = tf.keras.layers.Dense(1)(input)
  model = tf.keras.Model(inputs={"dense_feature": dense_input, "sparse_feature": sparse_input}, outputs=result)
  model.compile(optimizer, "mse", steps_per_execution=10)

이제 훈련 데이터세트로 모델을 훈련할 준비가 다 된 것입니다.

In [ ]:
model.fit(input_dataset, epochs=5, steps_per_epoch=10)

마지막으로 평가 데이터세트를 사용하여 모델을 평가합니다.

In [ ]:
model.evaluate(eval_dataset, steps=1, return_dict=True)

## 다음 단계

API 문서에서 TPU 전용 임베딩 설정에 대해 자세히 알아보세요.

- `tfrs.layers.embedding.TPUEmbedding`: 특히 특성 및 테이블 구성 관련, 옵티마이저 설정, 모델 생성(Keras [함수형](https://www.tensorflow.org/guide/keras/functional) API의 사용 또는 [하위 클래스화](../..guide/keras/custom_layers_and_models.ipynb) `tf.keras.Model`를 통해), 훈련/평가, `tf.saved_model`로 모델 제공
- `tf.tpu.experimental.embedding.TableConfig`
- `tf.tpu.experimental.embedding.FeatureConfig`

TensorFlow 2의 `TPUStrategy`에 대한 자세한 내용은 다음 리소스를 참조하세요.

- 가이드: [TPU 사용](../../guide/tpu.ipynb)(Keras `Model.fit`을 사용한 훈련/`tf.distribute.TPUStrategy`를 사용한 사용자 정의 훈련 루프 및 `tf.function`을 사용한 성능 개선 관련 팁 포함)
- 가이드: [TensorFlow를 사용하여 분산 훈련하기](../../guide/distributed_training.ipynb)
- 가이드: [TPUEstimator에서 TPUStrategy로 마이그레이션하기](tpu_estimator.ipynb)

훈련을 사용자 정의하는 방법에 대한 자세한 내용은 다음을 참조하세요.

- Guide: [Model.fit의 동작 사용자 정의하기](../..guide/keras/customizing_what_happens_in_fit.ipynb)
- 가이드: [훈련 루프 처음부터 작성하기](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)

Google의 머신러닝용 전문 ASIC인 TPU는 [Google Colab](https://colab.research.google.com/), [TPU Research Cloud](https://sites.research.google/trc/), [Cloud TPU](https://cloud.google.com/tpu)를 통해 사용할 수 있습니다.